# Chapter 02-02: Functional API

## 학습 목표
- Functional API로 비선형 구조의 모델을 만든다
- 잔차 연결(Residual Connection)을 구현한다
- 다중 출력 모델을 설계한다

## 목차
1. Functional API가 필요한 상황
2. 기본 사용법
3. Sequential과 동일한 MNIST 모델을 Functional로 재구현
4. 잔차 블록 구현
5. 다중 출력 모델

---
### 🐣 초등학생을 위한 Functional API 친절 설명!

#### ❓ Sequential API로 안 되는 경우가 있어요!

Sequential은 **레이어를 직선으로만** 쌓을 수 있어요.
하지만 실제 AI는 더 복잡한 구조가 필요해요!

| 필요 상황 | 예시 | Sequential 가능? |
|---------|------|---------------|
| **여러 입력** | 텍스트 + 이미지 동시 입력 | ❌ 불가 |
| **여러 출력** | 분류 + 회귀 동시 출력 | ❌ 불가 |
| **가지치기 구조** | 두 경로로 분기 후 합치기 | ❌ 불가 |
| **잔차 연결** | F(x) + x (ResNet 핵심) | ❌ 불가 |

#### 🏗️ Functional API 사용법 — 레고 블록처럼!

```python
# 1단계: 입력 정의 (재료 준비)
inputs = tf.keras.Input(shape=(784,))
# 2단계: 레이어를 함수처럼 호출 (블록 연결)
x = Dense(64, activation='relu')(inputs)
x = Dense(32, activation='relu')(x)
# 3단계: 출력 정의
outputs = Dense(10, activation='softmax')(x)
# 4단계: Model로 묶기
model = tf.keras.Model(inputs=inputs, outputs=outputs)
```

> 💡 **잔차 연결(Skip Connection)**이란?
> 입력 x를 변환하지 않고 **직접 더해주는** 지름길이에요!
> `출력 = F(x) + x` → 기울기가 직접 흐르는 길 생성 → 매우 깊은 네트워크 학습 가능!


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print("TensorFlow 버전:", tf.__version__)

## Functional API가 필요한 상황

- 다중 입력(Multiple Inputs): 텍스트 + 이미지 동시 입력
- 다중 출력(Multiple Outputs): 분류 + 회귀 동시 출력
- 잔차 연결(Residual Connection): Skip connection
- 가지치기(Branch) 구조

**수학적 기초 — Skip Connection**:
$$h_l = F(x_l, W_l) + x_l$$
레이어 입력을 출력에 직접 더함 → 기울기가 직접 흐르는 경로 생성

## 2. 기본 Functional API 사용법

Functional API는 `tf.keras.Input`으로 입력 텐서를 정의하고,
레이어를 **함수처럼 호출**하여 텐서를 연결한다.

In [ ]:
# ── 기본 Functional API 예시 ─────────────────────────────────────────────

# 1단계: 입력 텐서 정의 (배치 차원 제외)
inputs = tf.keras.Input(shape=(784,), name='input_layer')

# 2단계: 레이어를 함수처럼 호출하여 텐서 변환
x = tf.keras.layers.Dense(64, activation='relu', name='hidden_1')(inputs)  # inputs → x
x = tf.keras.layers.Dense(32, activation='relu', name='hidden_2')(x)       # x → x

# 3단계: 출력 텐서 정의
outputs = tf.keras.layers.Dense(10, activation='softmax', name='output')(x)

# 4단계: 입력과 출력 텐서를 묶어 Model 생성
func_model = tf.keras.Model(inputs=inputs, outputs=outputs, name='basic_functional')

func_model.summary()
print("\n입력 shape:", func_model.input_shape)
print("출력 shape:", func_model.output_shape)

## 3. Sequential과 동일한 MNIST 모델을 Functional API로 재구현

In [ ]:
# ── MNIST 분류 모델 — Functional 버전 ─────────────────────────────────────

# 28×28 이미지를 직접 입력으로 받음
img_inputs = tf.keras.Input(shape=(28, 28), name='image_input')

# Flatten: 2D 이미지 → 1D 벡터
x = tf.keras.layers.Flatten(name='flatten')(img_inputs)

# 은닉층
x = tf.keras.layers.Dense(128, activation='relu', name='hidden')(x)

# Dropout으로 과적합 방지
x = tf.keras.layers.Dropout(0.2, name='dropout')(x)

# 출력층
img_outputs = tf.keras.layers.Dense(10, activation='softmax', name='output')(x)

# 모델 인스턴스 생성
mnist_functional = tf.keras.Model(
    inputs=img_inputs,
    outputs=img_outputs,
    name='mnist_functional'
)

mnist_functional.summary()

# Sequential 모델과 파라미터 수 동일 여부 확인
print(f"\nFunctional 파라미터 수: {mnist_functional.count_params():,}")

## 4. 잔차 블록 구현

잔차 연결은 $h_l = F(x_l) + x_l$ 구조로,
입력을 변환 결과에 직접 더하여 **기울기 소실 문제**를 완화한다.

단, 더하기 위해서는 입출력 차원이 같아야 한다.

In [ ]:
# ── 잔차 블록을 포함한 모델 ───────────────────────────────────────────────

def residual_block(x, units, name_prefix):
    """잔차 블록: F(x) + x
    
    입력과 출력의 차원이 동일해야 덧셈이 가능하다.
    """
    # 변환 경로: Dense → Dense
    h = tf.keras.layers.Dense(units, activation='relu',
                               name=f'{name_prefix}_dense1')(x)
    h = tf.keras.layers.Dense(units, activation=None,       # 활성화 함수 적용 전에 더함
                               name=f'{name_prefix}_dense2')(h)

    # 스킵 연결: 입력을 그대로 더함
    out = tf.keras.layers.Add(name=f'{name_prefix}_add')([h, x])

    # 더한 뒤 활성화 함수 적용
    out = tf.keras.layers.Activation('relu', name=f'{name_prefix}_relu')(out)
    return out

# 입력 정의
res_inputs = tf.keras.Input(shape=(784,), name='res_input')

# 차원을 64로 맞추는 투영 레이어
x = tf.keras.layers.Dense(64, activation='relu', name='projection')(res_inputs)

# 잔차 블록 2개 적층
x = residual_block(x, units=64, name_prefix='res1')
x = residual_block(x, units=64, name_prefix='res2')

# 출력층
res_outputs = tf.keras.layers.Dense(10, activation='softmax', name='output')(x)

# 잔차 모델 생성
residual_model = tf.keras.Model(inputs=res_inputs, outputs=res_outputs,
                                 name='residual_model')
residual_model.summary()
print(f"\n잔차 모델 파라미터 수: {residual_model.count_params():,}")

## 5. 다중 출력 모델

하나의 모델에서 여러 개의 출력을 동시에 생성할 수 있다.
예: 주 분류기 + 보조 분류기(보조 손실로 훈련 안정화)

In [ ]:
# ── 다중 출력 모델 예시: 주 출력 + 보조 출력 ─────────────────────────────

multi_inputs = tf.keras.Input(shape=(28, 28), name='image_input')

# 공유 특징 추출 경로
x = tf.keras.layers.Flatten(name='flatten')(multi_inputs)
x = tf.keras.layers.Dense(128, activation='relu', name='shared_1')(x)
x = tf.keras.layers.Dense(64, activation='relu',  name='shared_2')(x)

# ─ 주 출력: 10개 클래스 분류 ─
main_output = tf.keras.layers.Dense(10, activation='softmax',
                                     name='main_output')(x)

# ─ 보조 출력: 홀수/짝수 이진 분류 (보조 손실) ─
aux_branch  = tf.keras.layers.Dense(32, activation='relu', name='aux_dense')(x)
aux_output  = tf.keras.layers.Dense(1,  activation='sigmoid',
                                     name='aux_output')(aux_branch)  # 짝수=0, 홀수=1

# 다중 출력 모델 생성 (outputs를 리스트로 전달)
multi_output_model = tf.keras.Model(
    inputs=multi_inputs,
    outputs=[main_output, aux_output],
    name='multi_output_model'
)

multi_output_model.summary()

# 다중 출력 모델 컴파일: 출력별로 손실함수 지정
multi_output_model.compile(
    optimizer='adam',
    loss={
        'main_output': 'sparse_categorical_crossentropy',  # 주 출력 손실
        'aux_output':  'binary_crossentropy'               # 보조 출력 손실
    },
    loss_weights={
        'main_output': 1.0,   # 주 손실 가중치
        'aux_output':  0.3    # 보조 손실 가중치 (작게 설정)
    },
    metrics={'main_output': 'accuracy'}
)
print("\n다중 출력 모델 컴파일 완료")

## 정리

| 항목 | Sequential | Functional |
|------|-----------|------------|
| 구조 | 선형 (레이어 순차 연결) | 비선형 (임의 연결 가능) |
| 다중 입출력 | 불가 | 가능 |
| 잔차 연결 | 불가 | 가능 |
| 모델 시각화 | 가능 | 가능 |
| 사용 난이도 | 쉬움 | 중간 |
| 권장 상황 | 간단한 선형 모델 | 복잡한 아키텍처 |

**다음**: 03_subclassing_api.ipynb — 완전 커스텀 모델 구현